In [102]:
import math

from jupyterscad import view
from solid2 import cube, cylinder, rotate, translate

In [153]:
fn = 360

In [12]:
def make_cube(x: float, y: float, z: float):
    obj = cube(size=[x, y, z], center=True)
    obj = translate([0, 0, z / 2])(obj)
    return obj

In [141]:
def make_zaru_(height: float, width: float, thickness: float, zaru_width: float, slit_width: float):
    num_zaru = int(width // (slit_width + zaru_width))
    num_slit = int(math.ceil(width / (slit_width + zaru_width)))

    obj = None
    for n in range(num_zaru + 1):
        obj_ = make_cube(zaru_width, height, thickness)
        obj_ = translate([zaru_width / 2 + n * zaru_width + (n) * slit_width, 0, 0])(obj_)
        obj = obj + obj_ if obj is not None else obj_
    if num_zaru == num_slit:
        obj = translate([slit_width / 2 - width / 2, 0, 0])(obj)
    else:
        obj = translate([slit_width - width / 2, 0, 0])(obj)

    obj = obj * make_cube(width, height, thickness)
    return obj


def make_zaru(height: float, width: float, thickness: float, zaru_width: float, slit_width: float):
    obj = make_zaru_(height, width, thickness, zaru_width, slit_width)
    obj = obj + translate([0, 0, thickness])(
        rotate([0, 0, 90])(make_zaru_(width, height, thickness, zaru_width, slit_width))
    )
    return obj

In [155]:
base_thickness = 1.0
pipe_radius = 15.0
outer_radius = 26.0
height = 12.0

main_obj = cylinder(h=base_thickness, r=outer_radius, _fn=fn)

pipe_cylinder = cylinder(h=height, r=pipe_radius, _fn=fn)
main_obj = main_obj + pipe_cylinder

minus_cylinder = cylinder(h=height, r=pipe_radius - base_thickness, _fn=fn)
main_obj = main_obj - minus_cylinder

main_obj = main_obj + pipe_cylinder * translate([0, 0, height - base_thickness])(
    make_zaru(2 * pipe_radius, 2 * pipe_radius, base_thickness / 2, base_thickness, base_thickness)
)

view(main_obj)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.7, position=(3.0, 5.0,…